In [2]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Python path
import sys
base_folder = 'DataScience'
location_base = os.path.join(os.getcwd().split(base_folder)[0], base_folder)
location_module = [os.path.join(location_base, 'Module')] 
for each in location_module:
    if each not in sys.path:
        sys.path.append(each)

# Auto reload of library
%reload_ext autoreload
%autoreload 2

from module_KTX_KK import * 
from preprocessing_KK import *
from algorithm_machinelearning_KK import *
from algorithm_deeplearning_KK import *
from evaluation_KK import *

Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


# Data Preprocessing

In [4]:
# 데이터로딩
file_location=r'G:\내 드라이브\Research\SavedData\KoSubway\df_subway_prep.csv'
df_old = pd.read_csv(file_location)

In [17]:
# 데이터로딩
file_location=r'G:\내 드라이브\Research\SavedData\KoSubway\df_subway_prep_20241112.csv'
df = pd.read_csv(file_location)
print(df.shape)

# 전처리
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    
    # 수송인원 높은 역 필터링
    top_stations = ['홍대입구', '잠실', '강남', '서울역', '신림', '고속터미널', '신도림', '역삼', '선릉', 
                  '을지로입구', '종각', '가산디지털단지', '서울대입구', '성수', '혜화', '양재', 
                   '사당', '명동', '압구정', '수유', '연신내', '건대입구', '합정', '신사', '광화문', 
                    '시청', '여의도', '경복궁', '안국', '종로3가'] 
    
    #top_stations = ['서울대입구','선릉','성수','신림','신도림'] 
    
    df = df[df['역명'].isin(top_stations)].reset_index(drop=True)
    
    selected_columns = df.columns
    
    # 제외할 변수 지정
    excluded_columns = ['역명', '면적', '승강장혼잡도2', '승강장혼잡도1', '날짜', '수송인원수', 'L_total', 'λ_total',
                        'W_total', 'W_환승', 'λ_환승', 'W_승차', 'λ_승차', '승강장_혼잡도', '요일구분','승하차인원','일별_승하차인원_합','호선'
                        ,'시간대별_환승_승객_수','시간대별_승하차_비율']
    
    feature_columns = [col for col in selected_columns if col not in excluded_columns]
    
    # 독립 변수와 종속 변수 분리
    X = df[feature_columns]
    y = df['승강장_혼잡도']
    
    # 학습용과 테스트용 데이터로 분할
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    
    # 스케일링
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 결과를 딕셔너리 형태로 반환
    data = {
        'X_train': X_train,
        'Y_train': Y_train,
        'X_test': X_test,
        'Y_test': Y_test,
        'X_train_scaled': X_train_scaled,
        'X_test_scaled': X_test_scaled,
        'feature_columns': feature_columns,
        'scaler': scaler,  # 추가
        'df_train': df.iloc[X_train.index],
        'df_test': df.iloc[X_test.index]
    }
    
    return data

# 데이터 전처리
df = preprocess_data(file_location)

(1649016, 114)


In [21]:
df['df_train'][df['feature_columns']]

,시간,승차인원,하차인원,우대권인원수,청소년인원수,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),출입구,섬식여부,환승노선_개수,환승유입인원수,지하철요금,달러 환율,승하차인원차,보통휘발유,자동차용경유,실업률(%),청년실업률(%),동행지수 순환변동치,선행지수 순환변동치,서울_인구(천명),서울_인구밀도,수도권_인구(천명),수도권_인구밀도,자동차등록대수(만대),기준금리,한국CPI,CPI_대비_지하철요금,요일_Saturday,요일_Sunday,공휴일여부,요일_Monday,요일_Tuesday,요일_Wednesday,요일_Thursday,요일_Friday,년도_2021,년도_2022,년도_2023,월_1,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,일_1,일_2,일_3,일_4,일_5,일_6,일_7,일_8,일_9,일_10,일_11,일_12,일_13,일_14,일_15,일_16,일_17,일_18,일_19,일_20,일_21,일_22,일_23,일_24,일_25,일_26,일_27,일_28,일_29,일_30,일_31,상선 혼잡도,하선 혼잡도,상행_평균운행간격,하행_평균운행간격,상행_운행횟수,하행_운행횟수,역_시간별_평균운행간격,월_카운트,월평균_환승유입인원수
273620,7,81,292,70,9,21.2000,1.8000,1.2000,100,0.0000,9,1,0,"341,710.0000","1,250.0000","1,130.5300",-211,"1,610.7900","1,407.4400",3.2000,7.2000,99.4000,102.7000,9508,15709,26069,2196,2491,0.5000,2.6000,480.7692,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.9000,4.5000,6.0000,6.1100,8,9,6.0550,31,"11,022.9032"
787140,17,1802,3180,1265,252,24.5000,0.0000,2.7000,56,0.0000,0,0,1,"474,173.0000","1,250.0000","1,288.2300",-1378,"2,092.2000","2,096.1000",3.0000,6.9000,100.8000,100.7000,9421,15567,26086,2197,2550,1.7500,6.0000,208.3333,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,65.9000,20.6000,6.0000,4.3100,9,13,5.1550,30,"15,805.7667"
1234132,14,3236,5911,384,706,17.4000,0.0000,3.8000,37,0.0000,4,1,2,"1,190,635.0000","1,250.0000","1,316.3000",-2675,"1,620.6900","1,529.2300",2.8000,6.4000,100.5000,98.6000,9400,15533,26190,2206,2595,3.5000,3.7000,337.8378,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.0500,22.3500,4.5500,4.9100,11,11,4.7300,30,"39,687.8333"
1070860,5,18,25,18,0,-9.2000,0.0000,0.8000,73,1.6000,5,1,1,"350,755.0000","1,250.0000","1,279.5500",-7,"1,528.5800","1,733.8200",3.0000,5.2000,100.2000,99.6000,9421,15567,26086,2197,2550,3.2500,5.0000,250.0000,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6.3000,10.3000,6.0000,9.0000,3,3,7.5000,31,"11,314.6774"
24808,12,1017,679,390,50,-4.4000,0.0000,2.3000,35,0.0000,8,0,0,"407,372.0000","1,250.0000","1,103.2700",338,"1,445.3800","1,245.7700",5.7000,9.5000,98.1000,101.3000,9508,15709,26069,2196,2491,0.5000,0.6000,"2,083.3333",0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.0000,24.9500,4.5000,4.3100,12,13,4.4050,31,"13,141.0323"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152315,21,1171,92,109,12,8.0000,0.0000,4.1000,65,0.0000,4,1,2,"126,417.0000","1,250.0000","1,121.3700",1079,"1,534.9000","1,333.1700",4.0000,10.0000,99.4000,102.3000,9508,15709,26069,2196,2491,0.5000,2.3000,543.4783,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16.7000,31.4500,5.7800,5.7800,9,9,5.7800,30,"4,213.9000"
963395,9,637,3499,259,141,14.6000,0.0000,2.1000,78,0.0000,10,1,0,"540,772.0000","1,250.0000","1,430.6900",-2862,"1,666.3800","1,829.5400",2.4000,5.6000,101.2000,100.3000,9421,15567,26086,2197,2550,3.0000,5.7000,219.2982,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34.5500,45.8500,3.4100,4.0000,17,14,3.7050,31,"17,444.2581"
117952,19,390,129,54,12,5.3000,0.0000,5.0000,54,0.0000,4,1,2,"180,956.0000","1,250.0000","1,129.1200",261,"1,527.4000","1,326.4900",4.3000,10.0000,98.5000,101.9000,9508,15709,26069,2196,2491,0.5000,1.5000,833.3333,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0

In [22]:
df_old.shape, df['df_train'][df['feature_columns']].shape

((5571731, 49), (1319212, 94))

In [23]:
df['df_train'][df['feature_columns']].head()

,시간,승차인원,하차인원,우대권인원수,청소년인원수,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),출입구,섬식여부,환승노선_개수,환승유입인원수,지하철요금,달러 환율,승하차인원차,보통휘발유,자동차용경유,실업률(%),청년실업률(%),동행지수 순환변동치,선행지수 순환변동치,서울_인구(천명),서울_인구밀도,수도권_인구(천명),수도권_인구밀도,자동차등록대수(만대),기준금리,한국CPI,CPI_대비_지하철요금,요일_Saturday,요일_Sunday,공휴일여부,요일_Monday,요일_Tuesday,요일_Wednesday,요일_Thursday,요일_Friday,년도_2021,년도_2022,년도_2023,월_1,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,일_1,일_2,일_3,일_4,일_5,일_6,일_7,일_8,일_9,일_10,일_11,일_12,일_13,일_14,일_15,일_16,일_17,일_18,일_19,일_20,일_21,일_22,일_23,일_24,일_25,일_26,일_27,일_28,일_29,일_30,일_31,상선 혼잡도,하선 혼잡도,상행_평균운행간격,하행_평균운행간격,상행_운행횟수,하행_운행횟수,역_시간별_평균운행간격,월_카운트,월평균_환승유입인원수
273620,7,81,292,70,9,21.2000,1.8000,1.2000,100,0.0000,9,1,0,"341,710.0000","1,250.0000","1,130.5300",-211,"1,610.7900","1,407.4400",3.2000,7.2000,99.4000,102.7000,9508,15709,26069,2196,2491,0.5000,2.6000,480.7692,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.9000,4.5000,6.0000,6.1100,8,9,6.0550,31,"11,022.9032"
787140,17,1802,3180,1265,252,24.5000,0.0000,2.7000,56,0.0000,0,0,1,"474,173.0000","1,250.0000","1,288.2300",-1378,"2,092.2000","2,096.1000",3.0000,6.9000,100.8000,100.7000,9421,15567,26086,2197,2550,1.7500,6.0000,208.3333,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,65.9000,20.6000,6.0000,4.3100,9,13,5.1550,30,"15,805.7667"
1234132,14,3236,5911,384,706,17.4000,0.0000,3.8000,37,0.0000,4,1,2,"1,190,635.0000","1,250.0000","1,316.3000",-2675,"1,620.6900","1,529.2300",2.8000,6.4000,100.5000,98.6000,9400,15533,26190,2206,2595,3.5000,3.7000,337.8378,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.0500,22.3500,4.5500,4.9100,11,11,4.7300,30,"39,687.8333"
1070860,5,18,25,18,0,-9.2000,0.0000,0.8000,73,1.6000,5,1,1,"350,755.0000","1,250.0000","1,279.5500",-7,"1,528.5800","1,733.8200",3.0000,5.2000,100.2000,99.6000,9421,15567,26086,2197,2550,3.2500,5.0000,250.0000,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,6.3000,10.3000,6.0000,9.0000,3,3,7.5000,31,"11,314.6774"
24808,12,1017,679,390,50,-4.4000,0.0000,2.3000,35,0.0000,8,0,0,"407,372.0000","1,250.0000","1,103.2700",338,"1,445.3800","1,245.7700",5.7000,9.5000,98.1000,101.3000,9508,15709,26069,2196,2491,0.5000,0.6000,"2,083.3333",0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.0000,24.9500,4.5000,4.3100,12,13,4.4050,31,"13,141.0323"


In [27]:
# descriptive_KK(df, target='혼잡도')

# Performance

In [36]:
# 데이터로딩
file_location=r'C:\DataScience\[DEBA]\Project-SubwayPrediction\Result\Performance_DataWizardd.xlsx'
df_result1 = pd.read_excel(file_location, sheet_name='승강장혼잡도(1)', header=1)
df_result2 = pd.read_excel(file_location, sheet_name='승강장혼잡도(2)', header=1)
df_result3 = pd.read_excel(file_location, sheet_name='승하차인원', header=1)
display(df_result1.head(), df_result2.head(), df_result3.head())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,실제값,예측값,MSE,RMSE,MSPE,MAE,MAPE,MedAE,MedAPE
0,홍대입구,2,Linear Regression,0.0306,0.0280,0.0000,0.0058,0.0591,0.0041,0.1600,0.0030,12.2512
1,홍대입구,2,Random Forest,0.0306,0.0306,0.0000,0.0005,0.0002,0.0002,0.0065,0.0001,0.2841
2,홍대입구,2,XGBoost,0.0306,0.0300,0.0000,0.0019,0.0067,0.0014,0.0599,0.0011,4.2550
3,홍대입구,2,LightGBM,0.0306,0.0298,0.0000,0.0021,0.0081,0.0016,0.0662,0.0012,4.6653
4,홍대입구,2,CatBoost,0.0306,0.0295,0.0000,0.0028,0.0121,0.0021,0.0807,0.0016,6.2009


,Unnamed: 0,Unnamed: 1,Unnamed: 2,실제값,예측값,MSE,RMSE,MSPE,MAE,MAPE,MedAE,MedAPE
0,강남,2,Linear Regression,147.3001,112.1940,"1,859.2270",43.1188,0.0624,35.2662,0.2310,32.2724,23.0543
1,강남,2,Random Forest,147.3001,147.0958,1.7141,1.3092,0.0002,0.4214,0.0043,0.1667,0.1361
2,강남,2,XGBoost,147.3001,145.3534,43.7659,6.6156,0.0045,4.9141,0.0451,3.8655,3.2895
3,강남,2,LightGBM,147.3001,145.0956,49.5489,7.0391,0.0040,5.1641,0.0447,4.0562,3.4221
4,강남,2,CatBoost,147.3001,143.1525,71.6041,8.4619,0.0092,6.4568,0.0609,5.1889,4.3960


,Unnamed: 0,Unnamed: 1,Unnamed: 2,실제값,예측값,MSE,RMSE,MSPE,MAE,MAPE,MedAE,MedAPE
0,강남,2,Linear Regression,"7,438.1843","7,438.1843",0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,강남,2,Random Forest,"7,438.1843","7,437.6006",327.2532,18.0901,0.0000,6.2555,0.0007,2.0400,0.0381
2,강남,2,XGBoost,"7,438.1843","7,434.1895","31,485.2003",177.4407,0.0006,113.1077,0.0170,66.9238,1.2530
3,강남,2,LightGBM,"7,438.1843","7,435.5642","26,634.3028",163.2002,0.0003,98.9534,0.0134,53.4927,1.0379
4,강남,2,CatBoost,"7,438.1843","7,429.3595","34,531.2494",185.8259,0.0004,110.9192,0.0152,61.8338,1.1642
